# First End Point

```python
from fastapi import FastAPI
from typing import Dict

# FastAPI instance
app = FastAPI(title="First Endpoint")

@app.get("/")
async def first_endpoint() -> Dict[str, str]:
    """
    A coroutine to add a GET endpoint at the root path.

    Returns
    -------
    Dict[str, str]
        Response
    """
    return {"first": "endpoint"}
```

## Explanation

### Point of Interaction

The `app` is the main point of interaction.

### Path Operation

This script creates a *path operation*. 

The "path" refers to the last part of the url starting from the first `/`; e.g., for `https://example.com/items/foo`, the path would be `/items/foo`. A "path" is also commonly called an "endpoint" or a "route".

"Operation" means one of the HTTP methods. When building APIs, we use these specific HTTP methods to perform a specific action:

* POST: to create data
* GET: to read data
* PUT: to update data
* DELETE: to delete data

In the HTTP protocol, we can communicate to each path using one (or more) of these "methods".

**In OpenAPI, each of the HTTP methods is called an "operation".**

### Path Operation Decorator

The `@app.get("/")` tells FastAPI that the function to be decorated is tasked to handle requests that:

* Corresponds to the path "/"
* Use a a `get` operation

### Path Operation Function

The `first_endpoint` coroutine is called by FastAPI whenever it receives a request to the URL "/" using a `GET` operation.

In [2]:
!uvicorn scripts.endpoint:app


INFO:     Started server process [57356]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
^C
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [57356]

Aborted!


# Path Parameters

```python
from enum import Enum 
from typing import Dict
from fastapi import FastAPI

class CountryName(str, Enum):
    usa = "usa"
    japan = "japan"
    china = "china"
    
app = FastAPI(title="Country")

@app.get("/country/{country_name}")
async def get_country(country_name: CountryName) -> Dict[str, str]:
    """
    Path operation function to get a country's name.

    Parameters
    ----------
    country_name : CountryName
        Path parameter

    Returns
    -------
    Dict[str, str]
        Response
    """
    match country_name:
        case CountryName.usa:
            return {"country_name": country_name, "message": "Chicken Nuggets"}
        case CountryName.japan:
            return {"country_name": country_name, "message": "Samurai"}
        case CountryName.china:
            return {"country_name": country_name, "message": "Pandas"}
```

## Explanation

### Enum

If we have a path operation that receives a path parameter with a set of possible valid path parameter values, we can use a standard Python `Enum`.

From the [official documentation](https://docs.python.org/3/library/enum.html), an enumeration:

* is a set of symbolic names (members) bound to unique values

* can be iterated over to return its canonical (i.e. non-alias) members in definition order

* uses call syntax to return members by value

* uses index syntax to return members by name

### Path Parameter

The `Enum` can be used directly as a type annotation for the path parameter.

Interactive docs will show the pre-defined values correctly.

### Comparisons

**The value of the path parameter will be an enumeration member**.

This means we can use the `is` keyword to do comparisons, e.g. `path_parameter is Enum.member`.

We can also get the actual value (a `str` in the example above) using `enum_member.value`.


In [ ]:
!uvicorn scripts.path_parameters:app 


INFO:     Started server process [45068]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     127.0.0.1:51543 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:51543 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:51544 - "GET /country/usa HTTP/1.1" 200 OK
INFO:     127.0.0.1:51546 - "GET /country/china HTTP/1.1" 200 OK
^C
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [45068]

Aborted!


# Query Parameters

```python
from typing import Union, Dict
from fastapi import FastAPI

app = FastAPI(title="Multiple Path & Query Parameters")

@app.get("/users/{user_id}/items/{item_id}")
async def read_user_item(user_id: int, item_id: str, needy: str, q: Union[str, None] = None, short: bool = False) -> Dict[str, Union[int, str]]:
    """
    Path operation function to get a user's item.

    Parameters
    ----------
    user_id : int
        Path parameter
    item_id : str
        Path parameter
    needy : str
        This is a required query parameter
    q : Union[str, None], optional
        Query parameter, by default None
    short : bool, optional
        Query parameter, by default False

    Returns
    -------
    Dict[str, Union[int, str]]
        A dictionary containing the user_id, item_id, and the needy query parameter
    """
    item = {"item_id": item_id, "owner_id": user_id, "needy": needy}
    if q:
        item.update({'q': q})
    if not short:
        item.update({'description': "This is a long description"}) 
    return item
```

## Explanation

We can declare multiple path parameters and query parameters with default or no default values.

In [ ]:
!uvicorn scripts.query_parameters:app


INFO:     Started server process [52868]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     127.0.0.1:51985 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:51985 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:51986 - "GET /users/12/items/soccer?needy=japan&short=false HTTP/1.1" 200 OK
INFO:     127.0.0.1:52002 - "GET /users/12/items/soccer?needy=japan&short=true HTTP/1.1" 200 OK
INFO:     127.0.0.1:52003 - "GET /users/12/items/soccer?needy=japan&q=crhis&short=true HTTP/1.1" 200 OK
^C
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [52868]

Aborted!


# Request Body

```python
from typing import Union, Dict
from fastapi import FastAPI
from pydantic import BaseModel

class Item(BaseModel):
    # Required attributes
    name: str
    price: float
    # Optional attributes
    description: Union[str, None] = None
    tax: Union[float, None] = None

app = FastAPI(title="Request Body")

@app.post("/items/{item_id}")
async def create_item(item_id: int, item: Item, q: Union[str, None] = None) -> Dict[str, Union[str, int]]:
    result = {"item_id": item_id, **item.model_dump()}
    # If query parameter is given
    if q:
        result.update({"q": q})
    # If tax is specified, update   
    if result['tax']:
        result.update({'price_with_tax': result['price'] + result['tax']})
    return result
```

## Explanation

| Element         | Description                                                                                           | Details and Usage                                                                                                            |
|-----------------|-------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------|
| **Request Body**| Data sent by the client (e.g., a browser) to the API.                                                | - Not always required.                                                                                                       |
|                 |                                                                                                       | - Used to send necessary information to the API to perform a specific operation (e.g., POST data for creating an item).      |
|                 |                                                                                                       | - Declared using Pydantic models which offer robust data validation, serialization, and documentation benefits.              |
| **Response Body**| Data sent by the API back to the client.                                                             | - Almost always necessary.                                                                                                   |
|                 |                                                                                                       | - Provides the results of the requested operation by the API to the client (e.g., status of the request, errors, returned data).|
| **Pydantic Models**| Used to define data structures for both request bodies and response payloads.                        | - Enhances data integrity through strict type checking.                                                                      |
|                 |                                                                                                       | - Simplifies data conversion between the API and Python data types, and automates error handling.                            |
|                 |                                                                                                       | - Includes automatic JSON schema generation for API documentation (with FastAPI).                                      |


### FastAPI 

By using Pydantic models, FastAPI will:

* JSON Reading and Conversion: FastAPI reads the incoming request body as JSON. It then converts this JSON data into the Python types specified in the Pydantic model, handling any necessary data type transformations.
  
* Data Validation: The framework validates the incoming data based on the model specifications. If any data is incorrect or invalid, FastAPI automatically generates a detailed error response. This error will clearly point out where the invalid data was found and what the issue was, improving *debuggability*.

* Parameter Utilization: After validation, the cleaned and verified data is made available in the function parameter (e.g., item), exactly as we declared it in the function's signature. This means we can interact with the data as a strongly typed object, enhancing code reliability and type-safety.

* Schema Generation: FastAPI generates [JSON Schema](https://json-schema.org/) definitions for the Pydantic models. These schemas are not only useful within the API but can also be utilized in other parts of the project where they might be relevant.

In [3]:
!uvicorn scripts.request_body:app


INFO:     Started server process [63720]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     127.0.0.1:53030 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53030 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:53031 - "POST /items/12?q=hi HTTP/1.1" 200 OK
^C
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [63720]

Aborted!
